# 

<a target="_blank" href="https://colab.research.google.com/github/sakarimov/Basic-ML-Model/blob/main/CNN.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import tensorflow as tf
print(tf.__version__)

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

IN_COLAB

In [ ]:
import zipfile, os

local_zip = (
    'datasets/images/archive.zip' if IN_COLAB == False
    else '/content/drive/MyDrive/datasets/images/archive.zip'
)
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('datasets/images/messy_clean_room/')
zip_ref.close()

base_dir = 'datasets/images/messy_clean_room/images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [ ]:
os.listdir(base_dir+'/train')

In [ ]:
os.listdir(base_dir+'/val')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
    rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 4,
    class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (150,150),
    batch_size = 4,
    class_mode = 'binary')

In [ ]:
dir(train_generator)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape = (150, 150, 3)),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape = (150, 150, 3))),
model.add(tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(512, (3,3), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy',
              optimizer = tf.optimizers.Adam(),
              metrics = ['accuracy'])

model.fit(
    train_generator,
    epochs = 40,
    validation_data = validation_generator,
    validation_steps = 5,
    verbose = 2)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2
transfer = tf.keras.models.Sequential([
    ResNet152V2(
        weights = "imagenet", 
        include_top = False, 
        input_tensor = tf.keras.layers.Input(shape = (150,150,3))),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'softmax')
])

In [ ]:
transfer.compile(loss = 'binary_crossentropy',
              optimizer = tf.optimizers.Adam(),
              metrics = ['accuracy'])

In [ ]:
transfer.fit(
    train_generator,
    epochs = 40,
    validation_data = validation_generator,
    validation_steps = 5,
    verbose = 2)